### download the CanSIPSv2 hindcast dataset from the IRI 

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt

In [2]:
import os 
import sys 
import pathlib

In [3]:
from datetime import datetime, timedelta
import calendar
from dateutil import relativedelta

In [4]:
import numpy as np 
import pandas as pd 
import xarray as xr

In [5]:
CWD = pathlib.Path.cwd()

In [6]:
HOME = pathlib.Path.home()

In [7]:
GCM = 'CanSIPSv2'

In [8]:
data_stream = 'FORECAST'

In [9]:
dict_variables = {}
dict_variables['PRECIP'] = 'prec'
dict_variables['T2M'] = 'tref'
dict_variables['SST'] = 'sst'
dict_variables['Z500'] = 'hgt'

In [10]:
dict_coords = {}
dict_coords['L'] = 'step'
dict_coords['M'] = 'member'
dict_coords['S'] = 'time'
dict_coords['X'] = 'lon'
dict_coords['Y'] = 'lat'

In [11]:
# for variable in ['T2M']: 
for variable in ['PRECIP','T2M', 'SST', 'Z500']: 
    
    opath = HOME / 'drives' / 'auck_projects' / 'END19101' / 'Working' / 'data' / 'hindcasts' / 'IRI' / GCM.replace('-','_') / variable
    
    if not opath.exists():
        opath.mkdir(parents=True)
    
    url = f'http://iridl.ldeo.columbia.edu/SOURCES/.Models/.NMME/.{GCM}/.{data_stream}/.MONTHLY/.{dict_variables[variable]}/dods'
    
    dset = xr.open_dataset(url, decode_times=False)
    
    if variable == 'Z500': 
        dset = dset.sel(P2=500).drop('P2')
    
    dset = dset.rename(dict_coords)
    dset = dset.rename({dict_variables[variable]:variable.lower()})
    
    origin = datetime(1960,1,1)
    dates = [origin + relativedelta.relativedelta(months=x) for x in dset.time]
    dset['time'] = (('time'), dates)
    
    # here select the time period 
    dset = dset.sel(time=slice('1998-09-01', None))
    
    for date in dset.time: 
        sub = dset.sel(time=date)
        date = pd.to_datetime(date.data)
        print("saving {} for {:%Y-%m}".format(variable, date))
        sub.to_netcdf(opath / f'{GCM}_hindcasts_{variable}_{date.year}_{date.month}.nc')
        sub.close()
        
    dset.close() 
    

saving PRECIP for 2019-08
saving PRECIP for 2019-09
saving PRECIP for 2019-10
saving PRECIP for 2019-11
saving PRECIP for 2019-12
saving T2M for 2019-08
saving T2M for 2019-09
saving T2M for 2019-10
saving T2M for 2019-11
saving T2M for 2019-12
saving SST for 2019-08
saving SST for 2019-09
saving SST for 2019-10
saving SST for 2019-11
saving SST for 2019-12
saving Z500 for 2019-08
saving Z500 for 2019-09
saving Z500 for 2019-10
saving Z500 for 2019-11
saving Z500 for 2019-12


### alternative using NCKS 

```
ncks -d S,264.,264. http://iridl.ldeo.columbia.edu/SOURCES/.Models/.NMME/.NCEP-CFSv2/.HINDCAST/.MONTHLY/.prec/dods ess.nc
```